In [1]:
import os
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

In [3]:
!rm -rf bert
!git clone https://github.com/google-research/bert

import sys

sys.path.append('bert/')

import codecs
import collections
import json
import re
import os
import pprint
import numpy as np
import tensorflow as tf
import modeling
import tokenization
import numpy as np
import pandas as pd
import re, string, unicodedata
import os
from os import path

#Visualization Libs
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# NLP Libs
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from bs4 import BeautifulSoup
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Additional Libs
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

# Deep Learning Libs
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout


Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 4.32 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [ ]:
import os

if os.environ['COLAB_TPU_ADDR']:
  cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
  strategy = tf.distribute.TPUStrategy(cluster_resolver)
  print('Using TPU')
elif tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
  print('Using GPU')
else:
  raise ValueError('Running on CPU is not recommended.')

In [5]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.6 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 6.6 MB 31.3 MB/s 
     |████████████████████████████████| 596 kB 33.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import torch
import time
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv('/content/gdrive/MyDrive/fake_or_real_news.csv')

MessageError: ignored

In [9]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [10]:
df = df[['title','text','label']]
null_df = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending = False) * 100

null_df = pd.concat([null_df, percent], axis = 1, keys = ['Counts', '% Missing'])
null_df.head()

,Counts,% Missing
title,0,0.0
text,0,0.0
label,0,0.0


In [11]:
true = df[df['label'] == 'REAL'][:2000]
fake = df[df['label'] == 'FAKE'][:2000]

true.head()

,title,text,label
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
7,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [12]:
true.shape, fake.shape # relative same size good for training model

((2000, 3), (2000, 3))

In [13]:
true['label'] = 1
fake['label'] = 0

df = pd.concat([true, fake],ignore_index= True).drop_duplicates()
df.sample(10)

,title,text,label
3793,Comment on Man of Sin: Obama says illegal alie...,"Posted on November 6, 2016 by Dr. Eowyn | 9 Co...",0
342,OnPolitics | 's politics blog,What you need to know about the election recou...,1
2037,BREAKING : DOJ Says They Will “HELP” Review th...,BREAKING : DOJ Says They Will “HELP” Review th...,0
3108,Devastating Wiki Leaks Show Scope Of Clinton C...,"Leave a reply \nOn the October 27, 2016 Fox Ne...",0
3664,Chinese Immigrants Demand Protection from Pari...,Chinese Immigrants Demand Protection from Pari...,0
3501,Comment on Will the Cost of a “Self-Driving Ta...,by Lambert Strether \nBy Lambert Strether of C...,0
1391,The new mess Republicans have created for them...,The Republican Party has put itself in an impo...,1
1463,We Republicans Lost On Gay Rights. That’s A Go...,I’m not among those Republicans who have “evol...,1
1684,Rubio Goes From Robot to Terminator Against Do...,Marco Rubio was on a mission on Thursday night...,1
2540,WikiLeaks Documents Reveal United Nations Inte...,WikiLeaks Documents Reveal United Nations Inte...,0


In [14]:
df['text'] = df['title'] + "" + df['text']
# delete the columns we do not need for the rest of the work
# del df['column name']
del df['title']

In [15]:

nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
def strip_html(text):
  # get a BeautifulSoup object that follows standard structure
  soup = BeautifulSoup(text, "html.parser")
  return soup.get_text() # extract all the text from a page

# removing the square brackets
def remove_between_square_brackets(text):
  # '' replace patterns occured in left of text like \[[^]*\]
  return re.sub(r'[[^]*\]', '', text)

# removing URL's
def remove_urls(text):
  return re.sub(r'http\S+', '', text)

# removing the stopwords from text
def remove_stopwords(text):
  final_text = []
  for i in text.split():
    if i.strip().lower() not in stop:
      final_text.append(i.strip()) # strip(None) remove blank char in default
  return " ".join(final_text)
  
# removing the noisy text
def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  text = remove_stopwords(text)
  return text

# Apply function on review column
df['text'] = df['text'].apply(denoise_text)

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Possible nested set at position 1
  if __name__ == '__main__':


,text,label
0,Kerry go Paris gesture sympathyU.S. Secretary ...,1
1,Battle New York: Primary MattersIt's primary d...,1
2,‘Britain’s Schindler’ Dies 106A Czech stockbro...,1
3,Fact check: Trump Clinton 'commander-in-chief'...,1
4,Iran reportedly makes new push uranium concess...,1


In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], stratify = df['label'])

In [67]:
max_len =  128
train_batch_size = 16
valid_batch_size = 16
epochs = 3
learning_rate = 1e-5
filters = 50
filter_sizes = [2,3 ,4 ,5]
hidden_size = 768
dropout = 0.1


In [63]:
from transformers import BertTokenizer, BertModel


In [64]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [65]:
x_train_tokens = tokenizer.batch_encode_plus(
    x_train.tolist(),
    add_special_tokens = True,
    max_length = max_len,
    padding = 'max_length',
    truncation = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

x_test_tokens = tokenizer.batch_encode_plus(
    x_test.tolist(),
    add_special_tokens = True,
    max_length = max_len,
    padding = 'max_length',
    truncation = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [66]:
x_train_tokens, x_test_tokens

({'input_ids': tensor([[  101,  2602,  2355,  ...,  2110,  1012,   102],
         [  101,  2178,  3078,  ...,  8398, 11967,   102],
         [  101,  1018,  2477,  ...,  1517,  3327,   102],
         ...,
         [  101, 19267,  4978,  ...,  2102,  1010,   102],
         [  101,  3514,  7597,  ...,  3068,  1012,   102],
         [  101,  8112,  1024,  ..., 19557,  3064,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])},
 {'input_ids': tensor([[  101,  6221,  8398,  ...,  3685,  7219,   102],
         [  101,  7615,  8872,  ...,  4691,  4139,   102

In [26]:
y_train_2d = [[x] for x in y_train.tolist()]
y_test_2d = [[x] for x in y_test.tolist()]

y_train_tensor = torch.tensor(y_train_2d)
y_test_tensor = torch.tensor(y_test_2d)

In [27]:
#DataLoaders -> data become iteral
batch_size = 16

train_data = TensorDataset(x_train_tokens['input_ids'], x_train_tokens['attention_mask'], y_train_tensor)
train_data_sampler = RandomSampler(train_data)
train_data_loader = DataLoader(train_data, sampler = train_data_sampler, batch_size = batch_size)

test_data = TensorDataset(x_test_tokens['input_ids'], x_test_tokens['attention_mask'], y_test_tensor)
test_data_sampler = RandomSampler(test_data)
test_data_loader = DataLoader(test_data, sampler = test_data_sampler, batch_size = batch_size)



In [70]:
class bert_cnn_model(nn.Module):
  def __init__(self, model):
    super(bert_cnn_model, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased', return_dict = False)
    for param in self.bert.parameters():
      param.requires_grad = True #update data
    self.convs = nn.ModuleList([nn.Conv1d(1, filters, (k, hidden_size)) for k in filter_sizes])
    self.dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(filters * len(filter_sizes), 2)

  def conv_and_pool(self, x, conv):
    x = conv(x)
    x = nn.ReLU(x)
    x = x.squeeze(3)
    x = nn.MaxPool1d(x, x.size(2))
    x = x.squeeze(2)
    return x

  def forward(self, x):
    context = x[0]
    mask = x[2]

    encoder_out, pooled = self.bert(context, attention_mask = mask, output_all_encoded_layers=False)
    out = torch.cat([self.conv_and_pool(encoder_out,conv) for conv in self.convs],1)
    out = self.fc(out)
    return out
    

In [74]:
bc_model = bert_cnn_model(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bc_model.to(device)

In [76]:
#loss function
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes = np.unique(y_train), y = y_train)
weights = torch.tensor(class_weights, dtype = torch.float)
weights = weights.to(device)

cross_entropy = nn.NLLLoss(weight = weights)


In [77]:
class_weights

array([0.99833222, 1.00167336])

In [82]:
#optimizer
optimizer = torch.optim.Adam(params = bc_model.parameters(), lr = learning_rate)


In [85]:
#Train model

def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model, 
    optimizer
  ):

  valid_loss_min = np.Inf

  for epoch in range(1, n_epochs + 1):
      print("Epoch {:} / {:}".format(epoch, epochs))
      train_loss, valid_loss = 0, 0
      avg_train_accuracy, avg_valid_accuracy = 0, 0
      accuracy = 0
      model.train()
      train_predictions = []
      valid_predictions = []

      #training loop
      for idx, batch in enumerate(training_loader):
          batch = [r.to(device) for r in batch]
          #input_ids, attention_masks, labels = batch
          outputs = model(batch)
          model.zero_grad()  #clear previously calculated gradients
          optimizer.zero_grad()
          loss = cross_entropy(outputs, labels.squeeze(1))
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          train_loss = train_loss + (1/(idx + 1) * (loss.item() - train_loss))
          outs = outputs.detach().cpu().numpy()
          train_predictions.append(outs)
          predictions = torch.argmax(outputs, dim = 1)
          accuracy += sum([1 if predictions[i] == labels[i] else 0 for i in range(len(labels))])
          
      avg_train_accuracy = accuracy / len(training_loader.dataset)
      print(f'\Training Accuracy: {avg_train_accuracy}')
      print(f'\Training Loss: {train_loss: .f}')

      #validation loop
      accuracy = 0
      model.eval()
      with torch.no_grad():
        for idx, bacth in enumerate(validation_loader):
            batch = [r.to(device) for r in batch]
            #input_ids, attention_masks, labels = batch
            outputs = model(batch)
            loss = cross_entropy(outputs, labels.squeeze(1))
            valid_loss = valid_loss + (1/(idx + 1) * (loss.item() - valid_loss))
            outs = outputs.detach().cpu().numpy()
            valid_predictions.append(outs)
            predictions = torch.argmax(outputs, dim = 1)
            accuracy += sum([1 if predictions[i] == labels[i] else 0 for i in range(len(labels))])
            
      if valid_loss < valid_loss_min:
          valid_loss_min = valid_loss
          torch.save(model.state_dict(), 'saved_wrights.pt')

      avg_valid_accuracy = accuracy / len(validation_loader.dataset)
      print(f'\Validation Accuracy: {avg_valid_accuracy}')
      print(f'\Validation Loss: {valid_loss: .f}')
      #print(f'roc_auc score: {roc_auc_score(y_train_tensor, train_predictions)}')
      #print(f'F1 score: {f1_score(y_train_tensor, train_predictions)}')

  return bert_model
  


In [93]:
convs = nn.ModuleList([nn.Conv1d(1, filters, (k, hidden_size)) for k in filter_sizes])

In [96]:
ml = BertModel.from_pretrained('bert-base-uncased', return_dict = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
t, q = 0, 0
for idx, batch in enumerate(train_data_loader):
          batch = [r.to(device) for r in batch]
          print(batch[0], batch[1],batch[2])
          #input_ids, attention_masks, labels = batch
          #outputs = bc_model(batch)
          #print(outputs)
          t, q = batch[0], batch[2]
          break

In [97]:
ml(t, attention_mask = q, output_all_encoded_layers=False)

TypeError: ignored

In [86]:
trained_model = train_model(epochs, train_data_loader, test_data_loader, bc_model, optimizer)

Epoch 1 / 3


TypeError: ignored

In [28]:
model = BertModel.from_pretrained('bert-base-uncased', return_dict = False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
#freeze bert parameters
for param in model.parameters():
  param.requires_grad = False

In [30]:
import torch.nn as nn

class BERTClass(nn.Module):

  def __init__(self, model):
    super(BERTClass, self).__init__()
    self.model = model 
    self.dropout = nn.Dropout(0.3)
    self.relu = nn.ReLU()
    self.linear = nn.Linear(768, 2)
    #self.linear2 = nn.Linear(512, 2)
    #self.softmax = nn.LogSoftmax(dim = 1)

  def forward(self, input_ids, attention_mask):
    _, cls_hs = self.model(input_ids, attention_mask)
    output_linear = self.linear(cls_hs)
    output_relu = self.relu(output_linear)
    output = self.dropout(output_relu)
    #output = self.linear2(output_dropout)
    #output = self.softmax(output_dropout)
    return output


In [31]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
bert_model = BERTClass(model)
bert_model.to(device)

In [33]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

def build_model(bert_layer, max_len):
  input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
  attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
  token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")

  _, sequence_output = bert_layer([input_ids, attention_mask,token_type_ids])
  clf_output = sequence_output[:, 0, :]
  out = Dense(1, activation='sigmoid')(clf_output)

  model = tf.keras.models.Model(inputs=[input_ids, attention_mask,token_type_ids], outputs=out)
  model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [34]:
import tensorflow_hub as hub
%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 28.8 µs


In [35]:
x_train_tokens2 = tokenizer.batch_encode_plus(
    x_train.tolist(),
    add_special_tokens = True,
    max_length = max_len,
    padding = 'max_length',
    truncation = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

x_test_tokens2 = tokenizer.batch_encode_plus(
    x_test.tolist(),
    add_special_tokens = True,
    max_length = max_len,
    padding = 'max_length',
    truncation = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [ ]:
model = build_model(bert_layer, max_len = 160)

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 160)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     [(None, 1024),       335141889   ['input_ids[0][0]',              
                                 (None, 160, 1024)]               'attention_mask[0][0]',     

In [35]:
ids = np.array(x_train_tokens2['input_ids'])
mask = np.array(x_train_tokens2['attention_mask'])
token_ids = np.array(x_train_tokens2['token_type_ids'])

train_data2 = ids, mask, token_ids

In [ ]:
train_history = model.fit(
    train_data2, 
    np.array(y_train_tensor),
    validation_split=0.2,
    epochs=3,
    batch_size=16
)

Epoch 1/3


In [ ]:
n_epochs = 2
batch_size = 32
max_len = 160

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding, Bidirectional, LSTM, GlobalAveragePooling1D

def bert_cnn_model(bert_layer, max_len):
  input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
  attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
  token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")

  _, sequence_output = bert_layer([input_ids, attention_mask,token_type_ids])
  clf_output = sequence_output[:, 0, :]

  

  model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=outputs)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
  
  return model

In [ ]:
bc_model = bert_cnn_model(bert_layer, max_len = 160)

In [38]:
import tensorflow as tf
import tensorflow_hub as hub
!pip install tensorflow_text

In [39]:
import tensorflow_text as text

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [40]:
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

ed_train = get_sentence_embeding(x_train)
ed_test = get_sentence_embeding(x_test)

In [49]:
from keras.layers.merge import concatenate
from keras.layers import MaxPooling1D, Flatten, Dense, Conv1D, Embedding

# Bert Layers
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = "text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network Layers
embedding = tf.keras.layers.Dropout(0.1, name = 'dropout')(outputs['pooled_output'])

conv1 = Conv1D(filters = 50, kernel_size = 2, strides = 1, activation = 'relu')(embedding)
pooling1 = MaxPooling1D(pool_size = 1)(conv1)
layer1 = Flatten()(pooling1)

conv2 = Conv1D(filters = 50, kernel_size = 3, strides = 1, activation = 'relu')(embedding)
pooling2 = MaxPooling1D(pool_size = 1)(conv1)
layer2 = Flatten()(pooling2)

conv3 = Conv1D(filters = 50, kernel_size = 4, strides = 1, activation = 'relu')(embedding)
pooling3 = MaxPooling1D(pool_size = 1)(conv1)
layer3 = Flatten()(pooling3)

conv4 = Conv1D(filters = 50, kernel_size = 5, strides = 1, activation = 'relu')(embedding)
pooling4 = MaxPooling1D(pool_size = 1)(conv4)
layer4 = Flatten()(pooling4)

merged = concatenate([layer1, layer2, layer3, layer4])
out = Dense(1, activation = 'sigmoid', name = 'output')(merged)

# construct final model
model = Model(inputs = [text_input], outputs = out)

model.compile(loos = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


ValueError: ignored

In [44]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  